# Integrantes Grupo 2:
*   Diego Cheloni
*   Iván Ezequiel Fernandez
*   Jhon Alexander Ortiz
*   Jean Carlos Pita
*   Laura Gutierrez
*   María Pía Achigar
*   Pablo Romero Perez

# Proyecto Integrador: Predicción del desperdicio de alimentos a lo largo del tiempo

## Antecedentes sobre el Tema
La pérdida y el desperdicio de alimentos es un problema global significativo que impacta tanto la seguridad alimentaria como el medio ambiente. La FAO recopila datos sobre las cantidades de alimentos desperdiciados en diferentes etapas de la cadena de suministro. Utilizar series de tiempo para analizar y predecir estas pérdidas puede ayudar a implementar mejores estrategias para reducir el desperdicio. Fuente: https://www.fao.org/platform-food-loss-waste/flw-data/es/

## Disponibilidad de Datos e Infraestructura
El dataset proporcionado por la FAO contiene datos históricos sobre la pérdida y desperdicio de alimentos. Se cuenta con al menos 25 mil observaciones, lo que es adecuado para la aplicación de técnicas de series de tiempo. La infraestructura necesaria para realizar el análisis y entrenamiento de modelos está disponible en un entorno de Jupyter Notebook.

# Fases del proyecto
* Exploración de dataset
* ETL
  * Eliminación de duplicados
  * Eliminación features
  * Imputación de datos
  * Ajustar formato de features




## Exploración de dataset

### Carga de dependencias
Importación de librerías necesarias para el proyecto

In [1]:
import pandas as pd
import numpy as np
import re


### Utilidades
Esta sección contiene funciones reutilizadas durante el proyecto

In [2]:
# Función que me genera un resumen exploratorio de los datos
def data_quality(df, df_name):
    # Dimensión
    print("======= Dimensión ==========")
    print("")
    print(f"Las dimensiones de {df_name} son {df.shape}")
    print("")

    # Revisión de valores nulls
    print("======= Porcentaje de nulos ==========")
    print("")
    print(f"Porcentaje de nulos por columna en el dataset {df_name}:")
    print("")
    for colname in df.columns:
        col_nulls = np.round(((df[colname].isna().sum())*100)/len(df),2)
        print(f"{colname}: {col_nulls}")
    print("")

    # Revisión de formatos (dtypes)
    print("")
    print("======= Formatos ==========")
    print("")
    print(df.dtypes)

    # Revisión de duplicados
    print("")
    print("======= Cantidad de duplicados ==========")
    print("")
    print(f"Cantidad de duplicados en {df_name} : {df.duplicated().sum()}")

    # Revisión de valores únicos
    print("")
    print("======= Cantidad de valores únicos ==========")
    print("")
    for colname in df.columns:
        col_nunique = df[colname].nunique()
        print(f"{colname}: {col_nunique}")
    print("")


# Función para limpiar y formatear las columnas categoricas
def clean_format_category(category):
    # Eliminar caracteres especiales usando re
    category = re.sub(r'[^\w\s]', '', category)
    # Convertir a minúsculas
    category = category.lower()
    # Reemplazar espacios en blanco por guiones bajos
    category = re.sub(r'\s+', '_', category)
    return category

### Información general del Dataset:

El dataset a cargar contiene la siguiente información:
1. *m49_code*: Código numérico según la clasificación M49 de las Naciones Unidas para identificar países o áreas geográficas.
2. *country*: Nombre del país donde se recopilaron los datos.
3. *region*: Región específica dentro del país(en el dataset este campo parece estar vacío).
4. *cpc_code*: Código de la Clasificación Central de Productos de la ONU que especifica el tipo de producto.
5. *commodity*: Descripción del producto
6. *year*: Año en que se recogieron los datos.
7. *loss_percentage*: Porcentaje de pérdida del producto durante la etapa especificada, en forma decimal.
8. *loss_percentage_original*: Porcentaje de pérdida en formato porcentual como cadena de texto seguido por un signo de porcentaje (%).
9. *loss_quantity*: Cantidad de pérdida en kilogramos, expresada en texto con unidades.
10. *activity*: Actividad durante la cual se registró la pérdida
11. *food_supply_stage*: Etapa de la cadena de suministro de alimentos donde se registró la pérdida
12. *treatment*: Tratamiento o condiciones bajo las cuales se almacenó el producto
13. *cause_of_loss*: Causa de la pérdida
14. *sample_size*: Tamaño de la muestra utilizado en el estudio, si aplica
15. *method_data_collection*: Método utilizado para la recolección de datos
16. *reference*: Referencia a la fuente de los datos, proporcionando contexto y crédito al estudio o investigador.
17. *url*: URL relacionada con el estudio o datos, si aplica
18. *notes*: Notas adicionales sobre el registro, por ejemplo, "Reference has been generated automatically".
19. *month*: Número que representa el mes de año del registro
20. *periodo*: Fecha aproximada en que se agregan los registros

In [3]:
# Cargar el dataset
df = pd.read_csv('data_origen.csv')
df.sample(5)


C:\Users\lauri\AppData\Local\Temp\ipykernel_3096\333494490.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_origen.csv')


,m49_code,country,region,cpc_code,commodity,year,loss_percentage,loss_percentage_original,loss_quantity,activity,food_supply_stage,treatment,cause_of_loss,sample_size,method_data_collection,reference,url,notes,month,periodo
9703,404,Kenya,NaN,0113,Rice,2009,1.000000,1,NaN,Transportation,Transport,NaN,NaN,NaN,Modelled Estimates,NaN,https://www.aphlis.net/en/data/tables/value-ch...,NaN,9,2009-09-01
23838,840,United States of America,NaN,01212,Cabbages,2000,7.000000,7,NaN,NaN,Whole supply chain,NaN,NaN,NaN,National Accounts,NaN,https://www.ers.usda.gov/data-products/food-av...,NaN,5,2000-05-01
35708,840,Benin,NaN,112.0,Sorghum,2016,0.199877,2.65,NaN,Storage,Transport,NaN,NaN,NaN,Modelled Estimates,NaN,https://www.ers.usda.gov/data-products/food-av...,NaN,5,2016-05-01
18155,706,Somalia,NaN,0113,Rice,2004,2.500000,2.5,NaN,Winnowing,Farm,NaN,NaN,NaN,Modelled Estimates,NaN,https://www.aphlis.net/en/data/tables/value-ch...,NaN,4,2004-04-01
23878,840,United States of America,NaN,01330,Grapes,2000,9.000000,9,NaN,NaN,Whole supply chain,NaN,NaN,NaN,National Accounts,NaN,https://www.ers.usda.gov/data-products/food-av...,NaN,3,2000-03-01


In [4]:
data_quality(df, "df")

======= Dimensión ==========

Las dimensiones de df son (45416, 20)

======= Porcentaje de nulos ==========

Porcentaje de nulos por columna en el dataset df:

m49_code: 0.0
country: 0.0
region: 95.24
cpc_code: 0.0
commodity: 0.0
year: 0.0
loss_percentage: 0.0
loss_percentage_original: 0.0
loss_quantity: 97.88
activity: 11.03
food_supply_stage: 13.4
treatment: 94.77
cause_of_loss: 96.14
sample_size: 95.32
method_data_collection: 1.44
reference: 80.08
url: 12.7
notes: 91.06
month: 0.0
periodo: 0.0


======= Formatos ==========

m49_code                      int64
country                      object
region                       object
cpc_code                     object
commodity                    object
year                          int64
loss_percentage             float64
loss_percentage_original     object
loss_quantity                object
activity                     object
food_supply_stage            object
treatment                    object
cause_of_loss                object

---

## ETL

A continuación, realizamos una limpieza exhaustiva de las columnas para mejorar la calidad y relevancia de nuestro conjunto de datos. Para lograr esto, identificamos y eliminamos cualquier información redundante que no aportara valor significativo al análisis posterior. Además, nos enfocamos en las columnas que contenían datos faltantes (nulos) y llevamos a cabo la imputación de estos valores utilizando métodos adecuados para mantener la integridad y consistencia de los datos en esas columnas.

In [5]:
# Se realiza copia del dataframe original
data = df.copy()

### Eliminación features

Las siguientes features se eliminan debido a:

Que son una reexpresión de la feature *loss_percentage*
* loss_percentage_original
* loss_quantity

No son relevantes para el caso de uso que deseamos abordar
* method_data_collection
* reference
* url
* notes
* m49_code
* cpc_code

Que tiene un porcentaje alto de valores nulos y no es posible hacer una imputación confiable de datos
* region
* treatment
* cause_of_loss
* sample_size

In [6]:
columns_to_drop = ['region',
                    'loss_percentage_original',
                    'loss_quantity','treatment',
                    'cause_of_loss','sample_size',
                    'method_data_collection','reference',
                    'url','notes', 'm49_code', 'cpc_code']
df_drop = data.drop(columns_to_drop, axis=1)
df_drop.shape

(45416, 8)

###  Imputación de datos

Luego del análisis de calidad de los datos, determinamos que el criterio para la imputación sería identificar aquellas columnas que tuvieran más del 20% de valores faltantes. Durante este proceso, identificamos dos columnas que cumplían con este criterio: "activity" y "food_supply_stage", ambas clasificadas como variables categóricas. Con base en esta observación, optamos por imputar los valores faltantes utilizando la moda de cada columna, tomando en consideración el país al que pertenecían los datos.

In [7]:
moda_activity = df_drop["activity"].mode().iloc[0]
moda_food_supply_stage = df_drop["food_supply_stage"].mode().iloc[0]

moda_pais_activity = df_drop.groupby('country')['activity'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else moda_activity)
df_drop["activity"] = df_drop['activity'].fillna(moda_pais_activity)

moda_pais_stage = df_drop.groupby('country')['food_supply_stage'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else moda_food_supply_stage)
df_drop["food_supply_stage"] = df_drop["food_supply_stage"].fillna(moda_pais_stage)


### Eliminación de duplicados

In [8]:
data_no_dup = df_drop.drop_duplicates()
data_no_dup.shape

(45348, 8)

### Ajustar formato de features

In [9]:
# Se convierte la columna 'year' a formato de fecha, para poder realizar análisis de series de tiempo
data_no_dup.loc[:, 'year'] = pd.to_datetime(data_no_dup['year'], format='%Y')

C:\Users\lauri\AppData\Local\Temp\ipykernel_3096\2950886513.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2015-01-01 00:00:00', '2015-01-01 00:00:00', '2015-01-01 00:00:00',
 '2015-01-01 00:00:00', '2015-01-01 00:00:00', '2015-01-01 00:00:00',
 '2021-01-01 00:00:00', '2021-01-01 00:00:00', '2021-01-01 00:00:00',
 '2021-01-01 00:00:00',
 ...
 '2021-01-01 00:00:00', '2016-01-01 00:00:00', '2013-01-01 00:00:00',
 '2014-01-01 00:00:00', '2012-01-01 00:00:00', '2008-01-01 00:00:00',
 '2020-01-01 00:00:00', '2017-01-01 00:00:00', '2011-01-01 00:00:00',
 '2022-01-01 00:00:00']
Length: 45348, dtype: datetime64[ns]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_no_dup.loc[:, 'year'] = pd.to_datetime(data_no_dup['year'], format='%Y')


In [10]:
# Se aplica formato a las columnas categóricas
data_no_dup.loc[:, 'commodity'] = data_no_dup['commodity'].apply(clean_format_category)
data_no_dup.loc[:, 'food_supply_stage'] = data_no_dup['food_supply_stage'].apply(clean_format_category)
data_no_dup.loc[:, 'activity'] = data_no_dup['activity'].apply(clean_format_category)

In [11]:
# Con el objetivo de sacar provecho de estas variables categóricas, vamos a crear nuevas features con su frecuencia

data_food_clean = data_no_dup.copy()

# Contar la frecuencia de cada actividad, food_supply_stage y commodity
activity_counts = data_food_clean['activity'].value_counts()
food_supply_stage_counts = data_food_clean['food_supply_stage'].value_counts()
commodity_counts = data_food_clean['commodity'].value_counts()

# Crear nuevas columnas con la frecuencia de cada actividad, food_supply_stage y commodity
data_food_clean['frec_actividad_de_perdida'] = data_food_clean['activity'].map(activity_counts)
data_food_clean['frec_etapa_de_perdida'] = data_food_clean['food_supply_stage'].map(food_supply_stage_counts)
data_food_clean['frec_alimento'] = data_food_clean['commodity'].map(commodity_counts)

In [12]:
data_quality(data_food_clean, "data_food_clean")

======= Dimensión ==========

Las dimensiones de data_food_clean son (45348, 11)

======= Porcentaje de nulos ==========

Porcentaje de nulos por columna en el dataset data_food_clean:

country: 0.0
commodity: 0.0
year: 0.0
loss_percentage: 0.0
activity: 0.0
food_supply_stage: 0.0
month: 0.0
periodo: 0.0
frec_actividad_de_perdida: 0.0
frec_etapa_de_perdida: 0.0
frec_alimento: 0.0


======= Formatos ==========

country                              object
commodity                            object
year                         datetime64[ns]
loss_percentage                     float64
activity                             object
food_supply_stage                    object
month                                 int64
periodo                              object
frec_actividad_de_perdida             int64
frec_etapa_de_perdida                 int64
frec_alimento                         int64
dtype: object

======= Cantidad de duplicados ==========

Cantidad de duplicados en data_food_clean : 

In [13]:
data_food_clean.to_csv('data_food_clean.csv', index=False)

## La etapa de análisis continua en la notebook "2_Analisis.ipynb"

---